In [2]:

import torch
import os
from fairseq import checkpoint_utils
import fairseq



checkpoint_path = '/data/home/xwhan/fairseq-py/checkpoints/bart.large/model.pt'
state = checkpoint_utils.load_checkpoint_to_cpu(checkpoint_path)
state['cfg'].task.data = '/datasets01/bookwiki_CC-NEWS_openwebtext_stories-mmap2-bin/121219/bookwiki_CC-NEWS_openwebtext_stories-mmap2-bin'
task = fairseq.tasks.setup_task(state['cfg'].task)

# print(state['cfg'].model)
bart = task.build_model(state['cfg'].model)
bart.load_state_dict(state['model'], strict=True, model_cfg=state['cfg'].model)

bart_cfg = state['cfg']
bart_cfg.model.max_source_positions = 1024 * 16
bart_cfg.model.alibi = True
long_model = task.build_model(bart_cfg.model)

print(long_model)

# ##### encoder staff #####
# # 1. embed_tokens and layernorm_embedding
# long_model.encoder.embed_tokens.load_state_dict(bart.encoder.embed_tokens.state_dict())
# long_model.encoder.layernorm_embedding.load_state_dict(bart.encoder.layernorm_embedding.state_dict())

# # 2. attention layers
# long_model.encoder.layers.load_state_dict(bart.encoder.layers.state_dict(), strict=True)

# # 3. embed_positions, longer
# pos_limit, _ = bart.encoder.embed_positions.weight.shape
# new_pos_limit, embed_dim = long_model.encoder.embed_positions.weight.shape
# new_pos_embed = bart.encoder.embed_positions.weight.new_empty(new_pos_limit, embed_dim)
# step = pos_limit - 2
# for start in range(2, new_pos_limit, step):
#     new_pos_embed[start:start+step] = bart.encoder.embed_positions.weight[2:]
# long_model.encoder.embed_positions.weight.data = new_pos_embed

# ##### decoder staff #####
# long_model.decoder.load_state_dict(bart.decoder.state_dict())

# save_path = '/data/home/xwhan/fairseq-py/checkpoints/bart.large.block16k'
# task.dictionary.save(os.path.join(save_path, 'dict.txt'))
# state['args'] = bart_cfg.model
# state['model'] = long_model.state_dict()
# print(state.keys())
# if 'criterion' in state:
#     del state['criterion']
# state['extra_state'] = {"epoch": 0}
# state['last_optimizer_state'] = None
# torch.save(state, os.path.join(save_path, 'model.pt'))
